In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from warnings import filterwarnings

filterwarnings("ignore", category=DeprecationWarning) 
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")

In [ ]:
train.isnull().sum()

In [ ]:
train

In [ ]:
train.info()
# all float columns except id and target means no categorical columns

In [ ]:
x_train = train.drop(['id','target'],axis=1)
y_train = train['target']

In [ ]:
def detect_noisy_samples(X,y,thres=.5): #Returns noisy indexes
    rf = RandomForestClassifier(oob_score=True,n_jobs=-1).fit(X,y)
    noise_prob = 1 - rf.oob_decision_function_[range(len(y)),y]
    return noise_prob>thres

In [ ]:
%%time
noise = detect_noisy_samples(x_train,y_train)

In [ ]:
noise_idx =[]
for idx,boo in enumerate(noise):
    if boo:
        noise_idx.append(idx)

In [ ]:
# %percentage of noise in our dataset(It will have also added some clean data in it too)
print(f"noise = {len(noise_idx)/len(noise) *100}%")

In [ ]:
# removing noise rows
train.drop(noise_idx,inplace=True)

In [ ]:
train

In [ ]:
x_train = train.drop(['id','target'],axis=1)
y_train = train['target']

x_cols = x_train.columns
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(data = x_train, columns=x_cols)
x_train

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 0, stratify = y_train)

In [ ]:
X_train = X_train.to_numpy().reshape(X_train.shape[0],X_train.shape[1],1)
X_val = X_val.to_numpy().reshape(X_val.shape[0], X_val.shape[1], 1)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape = (100,1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid', kernel_regularizer=keras.regularizers.l1(0.01)))
model.compile(optimizer=Adam(lr=0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
checkpointer = [EarlyStopping(monitor = 'val_accuracy', verbose = 1, restore_best_weights=True,mode="max",patience = 9),
                ModelCheckpoint(
                    filepath='model.weights.best.hdf5',
                    monitor="val_accuracy",
                    verbose=1,
                    save_best_only=True,
                    mode="max")]

In [ ]:
#total epochs
epochs = 50

#model summary
model.summary()

In [ ]:
%%time
#training the model
history = model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_val, Y_val), verbose=1, callbacks=checkpointer)

In [ ]:
#saving model
model.save("best_model")

In [ ]:
# plotting curve of model accuracy over the training time
epochs = len(history.history['accuracy'])
epoch_range = range(1, epochs+1)
plt.plot(epoch_range, history.history['accuracy'])
plt.plot(epoch_range, history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
# plotting curve of model loss over the training time
plt.plot(epoch_range, history.history['loss'])
plt.plot(epoch_range, history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")
test_id = test.id
x_test = test.drop(['id'],axis=1)
# scaling the test data too
x_test = scaler.transform(x_test)

In [ ]:
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)

In [ ]:
pred = model.predict(x_test)
pred = pd.DataFrame(pred,columns=['target'])
sub = pd.concat([test_id,pred],axis=1)
sub.set_index('id',inplace=True)
sub.to_csv("Submission_file_noise_reduction.csv")